In [1]:
!pip install datasets
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 63.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.

In [2]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset

dataset = load_dataset("UC3M-LCPM/DIPROMATS_2024", "task1a")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10169 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/2179 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2180 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3', '__index_level_0__'],
        num_rows: 10169
    })
    val: Dataset({
        features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3', '__index_level_0__'],
        num_rows: 2179
    })
    test: Dataset({
        features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3', '_

In [5]:
dataset['train'] = dataset['train'].remove_columns(['language', 'test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3'])
dataset['val'] = dataset['val'].remove_columns(['language', 'test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3'])
dataset['test'] = dataset['test'].remove_columns(['language', 'test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3'])

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import numpy as np

## Entrenar modelo

In [7]:
model_id = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [9]:
labels_ids = {'False': 0, 'True': 1}

def preprocess_data(examples):
  text = list(examples['text'])

  model_inputs = tokenizer(text, padding="max_length", truncation=True, max_length=46)

  tokenized_ids = [labels_ids[str(label)] for label in examples["label_task1"]]

  model_inputs["labels"] = tokenized_ids

  return model_inputs

In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/10169 [00:00<?, ? examples/s]

Map:   0%|          | 0/2179 [00:00<?, ? examples/s]

Map:   0%|          | 0/2180 [00:00<?, ? examples/s]

In [11]:
encoded_dataset.set_format("torch")

In [12]:
from transformers import TrainingArguments, Trainer, AutoConfig

args = TrainingArguments(
    f"RoBERTa_base_binary_emoji_en_es",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    push_to_hub=True,
    hub_model_id="UC3M-LCPM/RoBERTa_base_binary_emoji_en_es",
)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [14]:
from huggingface_hub import HfFolder, notebook_login

In [15]:
repository_id = "UC3M-LCPM/RoBERTa_base_binary_emoji_en_es"
config = AutoConfig.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)

training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['val'],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.506300,0.423532,0.783387,0.822913,0.783387,0.795882
2,0.384300,0.388810,0.836163,0.838607,0.836163,0.837310
3,0.516100,0.451907,0.829738,0.841527,0.829738,0.834354
4,0.252400,0.650655,0.849013,0.847567,0.849013,0.848253
5,0.069300,0.717200,0.850849,0.845462,0.850849,0.847451


TrainOutput(global_step=6360, training_loss=0.33695577840150903, metrics={'train_runtime': 428.0669, 'train_samples_per_second': 118.778, 'train_steps_per_second': 14.857, 'total_flos': 1201929842424780.0, 'train_loss': 0.33695577840150903, 'epoch': 5.0})

In [18]:
model.push_to_hub("UC3M-LCPM/RoBERTa_base_binary_emoji_en_es")
tokenizer.push_to_hub("UC3M-LCPM/RoBERTa_base_binary_emoji_en_es")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/UC3M-LCPM/RoBERTa_base_binary_emoji_en_es/commit/e3ab7f0b3ad0491138d491e9c705b3a14fa4cceb', commit_message='Upload tokenizer', commit_description='', oid='e3ab7f0b3ad0491138d491e9c705b3a14fa4cceb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.evaluate()

## Predicciones

In [ ]:
import torch

In [ ]:
torch.cuda.empty_cache()

In [ ]:
labels_ids = {'False': 0, 'True': 1}
ids_labels = {0: 'False', 1: 'True'}

In [ ]:
from transformers import AutoModel

In [ ]:
model_ft = AutoModelForSequenceClassification.from_pretrained("UC3M-LCPM/RoBERTa_base_binary_emoji_en_es", num_labels=2, id2label=ids_labels, label2id=labels_ids)
tokenizer_ft = AutoTokenizer.from_pretrained("UC3M-LCPM/RoBERTa_base_binary_emoji_en_es")

In [ ]:
classifier = pipeline("sentiment-analysis", model=model_ft, tokenizer= tokenizer_ft)

In [ ]:
predictions = classifier(encoded_dataset['text'][:20])

In [ ]:
predictions

In [ ]:
inputs = tokenizer_ft(dataset['test']['text'][:20], return_tensors="pt", padding=True)
with torch.no_grad():
    logits = model_ft(**inputs).logits

predicted_class_id = logits.argmax().item()
model_ft.config.ids_labels[predicted_class_id]